In [ ]:
import time
import os
import winreg
import psutil
import vdf

CHECK_INTERVAL = 60
TOTAL_TIME = 5 * 60


def get_steam_path():
    try:
        key = winreg.OpenKey(
            winreg.HKEY_CURRENT_USER,
            r"Software\Valve\Steam"
        )
        steam_path, _ = winreg.QueryValueEx(key, "SteamPath")
        return steam_path.replace("/", "\\")
    except Exception:
        return None


def get_downloading_game(steam_path):
    steamapps = os.path.join(steam_path, "steamapps")
    for file in os.listdir(steamapps):
        if file.startswith("appmanifest") and file.endswith(".acf"):
            with open(os.path.join(steamapps, file), encoding="utf-8") as f:
                data = vdf.load(f)
                state = data["AppState"]
                if state.get("StateFlags") == "1026":
                    return state.get("name"), state.get("appid")
    return None, None

def get_download_speed(prev, curr, interval):
    return (curr - prev) / interval / 1024 / 1024

def monitor():
    steam_path = get_steam_path()
    if not steam_path:
        print("Steam не найден")
        return
    print("Steam найден:", steam_path)
    net = psutil.net_io_counters()
    prev_recv = net.bytes_recv
    for minute in range(1, 6):
        time.sleep(CHECK_INTERVAL)
        game, appid = get_downloading_game(steam_path)
        net = psutil.net_io_counters()
        speed = get_download_speed(prev_recv, net.bytes_recv, CHECK_INTERVAL)
        prev_recv = net.bytes_recv
        if game:
            status = "Пауза" if speed < 0.05 else "Загружается"
            print(f"[{minute}/5] {game} ({appid})")
            print(f"Статус: {status}")
            print(f"Скорость: {speed:.2f} MB/s\n")
        else:
            print(f"[{minute}/5] Загрузка не обнаружена\n")

if __name__ == "__main__":
    monitor()